## 023-Longest Increasing Subsequence

---

#### 遺伝子順序類似度の単純な指標
「Enumerating Gene Orders」では、2つの異なる種から得られた染色体上の遺伝子の順序を比較し、それらが進化の過程でどのように再配列されたかについて議論を始めた。  

2つの染色体上の遺伝子を比較する非常に単純な方法のひとつは、両方の染色体で同じ順序に見られる遺伝子の最大集合を探すことである。そのためには置換の概念を使う必要がある。例えば、2つの染色体が n 個の遺伝子を共有しているとする。1つ目の染色体の遺伝子を出現順に 1 から n でラベル付けすると、2つ目の染色体はこれらの番号の置換として表される。このとき、同じ順序で現れる遺伝子の最大数を求めるには、その置換における**最長増加部分列**を探せばよい。  

---

#### 問題
置換の部分列とは、その要素を出現順に並べた要素の集合である。  
例えば、(5, 1, 3, 4, 2) の部分列として (5, 3, 4) がある。  

部分列が「増加部分列」であるのは、要素が昇順になっている場合であり、「減少部分列」であるのは、要素が降順になっている場合である。  
例えば、置換 (8, 2, 1, 6, 5, 7, 4, 3, 9) において、増加部分列は (2, 6, 7, 9)、減少部分列は (8, 6, 5, 4, 3) である。これらが最長であることを確認できる。  

---

#### 入力  
- 正の整数 n (n ≤ 10000)  
- 長さ n の置換 π  

#### 出力  
- π の最長増加部分列  
- 続いて π の最長減少部分列  

---

#### サンプルデータセット
```
5
5 1 4 2 3
```

#### サンプル出力
```
1 2 3
5 4 2
```

In [ ]:
# 最長増加(または非減少)部分列を O(n log n) で復元
from bisect import bisect_left


def lis(array: list[int]) -> list[int]:
    n = len(array)
    tails = []  # それぞれの山の一番上の値（山の最小値）
    tails_idx = []  # それぞれの山の一番上の値が「元の配列のどのインデックス由来か」を記録
    parent = [-1] * n  # 特定のカードの「一つ前のカードが元の配列のどのインデックス由来か」を記録 （前が存在しないなら`-1`）

    for i, x in enumerate(array):
        pos = bisect_left(tails, x)
        if pos == len(tails):
            tails.append(x)
            tails_idx.append(i)
        else:
            tails[pos] = x
            tails_idx[pos] = i
        parent[i] = tails_idx[pos - 1] if pos > 0 else -1

    # 復元
    idx = tails_idx[-1]
    out = []
    while idx != -1:
        out.append(array[idx])
        idx = parent[idx]
    return out[::-1]


def lds(array: list[int]) -> list[int]:
    # LDS は符号反転して LIS を取る
    res = lis([-x for x in array])
    return [-x for x in res]


a = [5, 1, 4, 2, 3]
print(*lis(a))  # -> [1,2,3]
print(*lds(a))  # -> [5,4,2]

1 2 3
5 4 3


## 考え方

この問題は「カードを左から見て、できるだけ左の山へ置く」ソリティアのルールで考えられます。

最長増加部分列（LIS）を求めるために必要な考え方は大きく2段階です。

## 1. 山の数を考えると、LISの長さを得られます

なぜ山の数がLISの長さになるのか？

### 山の作り方のルール

- あるカードを一番左に置ける山の上に置く。  
- もしどの山にも置けなければ、新しい山を右端に作る。  

これを守ると、左から右へ行くほど山のトップは大きくなるという性質が保たれます。  
（つまり、山番号が大きいほど“末尾が大きい増加列”）

### 山1 → 山2 → … → 山k と辿ると必ず「増加部分列」になる

- 山1から1枚取る → 山2から1枚取る → … → 山kから1枚取る  
  という選び方をすれば、必ず単調増加の列を作ることができます。  

---

### 実例で確認（a = [5, 1, 4, 2, 3]）
カードを順に置いていくと：

- 5 → 山1: [5]  
- 1 → 山1を置き換え: [1]  
- 4 → 山2: [1][4]  
- 2 → 山2を置き換え: [1][2]  
- 3 → 山3: [1][2][3]  

最終的に 山が3つ。  
→ LIS の長さも 3。  
→ 実際の LIS = [1, 2, 3]。  

## 2. 右端の山のトップのカードの由来をたどると、LISが復元できます

- 山の数だけで 「LISの長さ」は分かりますが、「LISの中身」は、山のトップの値だけではわかりません。  
- そこで **インデックス情報** と **親ポインタ** を残しておく必要があります。  


### 復元の仕組み

1. **`tails_idx`**  
   - 各山のトップが **元の配列のどの位置**かを記録する。  
   - 例: `tails_idx[2] = 4` → 「3番目の山の一番上にあるカードは、元の配列の4番目の要素」。  

2. **`parent`**  
   - 各カードが「どのカードのあとに続くか」を記録する。  
   - カード `i` が「`pos+1`番目の山のトップ」になったとき、  
     カード`i`の前は必ず「 `pos`番目の山のトップのカード」＝`tails_idx[pos-1]`となる。  
   - よって、カード`i`の親は`tails_idx[pos-1]`となり、 `parent[i] = tails_idx[pos-1]` としてリンクを張る。  

3. **復元の流れ**  
   - `tails_idx[-1]`（一番右の山のトップ）からスタート。  
   - `parent` を辿っていけば「前の要素 → さらに前の要素 …」とたどれる。  
   - 最後に逆順に並べ直すと LIS が完成する。  


## 実例で確認（`tails` と LIS が異なる場合）

配列 `a = [3, 10, 2, 1, 13]` を考える。


### ステップごとの処理

初期状態  
- `tails=[]`, `tails_idx=[]`, `parent=[-1,-1,-1,-1,-1]`

1. `x=3`  
- 新しい山を作る  
- `tails=[3]`, `tails_idx=[0]`, `parent[0]=-1`

2. `x=10`  
- 3より大きいので新しい山を作る  
   - 新しい山を作るので、以下の操作をする
     - tails_idxに現在のカードのインデックス`1` を追加
     - parentに現在のカードのインデックス`1`と、その一つ前の山のトップのインデックス`0`を記録  
- `tails=[3,10]`, `tails_idx=[0,1]`, `parent[1]=0`

3. `x=2`  
- `tails` の中で置き換え → 3 の山を 2 に更新  
- `tails=[2,10]`, `tails_idx=[2,1]`, `parent[2]=-1`

4. `x=1`  
- `tails` の中で置き換え → 2 の山を 1 に更新  
- `tails=[1,10]`, `tails_idx=[3,1]`, `parent[3]=-1`

5. `x=13`  
- 10より大きいので新しい山を作る  
   - 新しい山を作るので、以下の操作をする
     - tails_idxに現在のカードのインデックス`4` を追加
     - parentに現在のカードのインデックス`4`と、その一つ前の山のトップのインデックス`1`を記録  
- `tails=[1,10,13]`, `tails_idx=[3,1,4]`, `parent[4]=1`


### 結果

- 元の配列： `a = [3, 10, 2, 1, 13]`

- `tails = [1,10,13]`  
- `tails_idx = [3,1,4]`  
- `parent = [-1,0,-1,-1,1]`

> [!NOTE]
> `parent`の値は「元の配列における前の要素のインデックス」  
> - `-1`: 最初の要素（前がない）
> - `0`: 元の配列の0番目（値=3）
> - `1`: 元の配列の1番目（値=10）


### LIS 復元
1. 開始：`idx = tails_idx[-1] = 4`（値=13）  
2. `parent[4] = 1` → 値=10  
3. `parent[1] = 0` → 値=3  
4. `parent[0] = -1` → 終了  

復元した LIS = `[3, 10, 13]`🎊🎊🎊


## 参考資料

https://www.cs.princeton.edu/courses/archive/spring13/cos423/lectures/LongestIncreasingSubsequence.pdf